# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调

这个 Notebook 会自动配置环境、启动微调 (QLoRA)、运行测试、合并 LoRA 适配器并导出 GGUF 模型。

**步骤：**
1. 克隆/更新 GitHub 仓库并定义路径。
2. 安装必要的依赖项。
3. 运行训练脚本 (QLoRA)。
4. (可选) 运行测试脚本与微调后的模型(适配器)交互。
5. 合并 LoRA 适配器到基础模型。
6. (可选) 将合并后的模型转换为 GGUF 格式。

In [ ]:
# 1. 克隆/更新仓库并定义路径
import os
repo_path = '/content/echoheart_demo'

# --- 配置变量 (硬编码) ---
base_model_name = "Qwen/Qwen2.5-1.5B-Instruct" 
adapter_path = "/content/echoheart_demo/output/qwen2-1.5b-qlora-ft" 
merged_model_path = "/content/echoheart_demo/output/qwen2-1.5b-merged-ft" 
gguf_output_file = f"{merged_model_path}/gguf-model-f16.gguf" 
# ---------------

if not os.path.exists(repo_path):
  print(f"Cloning repository into {repo_path}...")
  !git clone https://github.com/shuakami/echoheart_demo.git {repo_path}
else:
  print(f"Repository already exists at {repo_path}.")

%cd {repo_path}

print("Pulling latest changes...")
!git pull origin master

!pwd

In [ ]:
# 2. 安装依赖
print("Installing dependencies from requirements.txt...")
!pip install -q -r requirements.txt

print("Applying dependency fixes...")
!pip install fsspec==2024.12.0

In [ ]:
# 3. 运行训练脚本
print("Starting training...")
!python train.py

## 训练完成！

微调后的模型保存在 Colab 环境文件系统的 `output/qwen-ft` 目录中。

In [ ]:
# 4. 运行测试脚本
print("Starting non-interactive testing session...")
!python test_model.py --base_model_name Qwen/Qwen2.5-1.5B-Instruct --adapter_path output/qwen2-1.5b-qlora-ft 

## 5. (可选) 转换为 GGUF 格式

**注意:** 直接转换包含 LoRA 适配器的模型目录到 GGUF 通常**不可行**或效果不佳。`llama.cpp` 的转换脚本期望一个完整的模型目录。
理想情况下，应先执行**模型合并**步骤，将 LoRA 适配器合并回基础模型，保存为一个新的完整模型目录，然后再对合并后的目录进行 GGUF 转换。
以下代码**尝试直接转换适配器目录**（通常会失败），如果需要执行合并，请告知。
转换后的文件将尝试保存在 `{gguf_output_file}`。

In [ ]:
# 运行 LoRA 合并脚本
print("Starting LoRA merge...")
!python merge_lora.py --base_model_name "Qwen/Qwen2.5-1.5B-Instruct" --adapter_path "/content/echoheart_demo/output/qwen2-1.5b-qlora-ft" --output_path "/content/echoheart_demo/output/qwen2-1.5b-merged-ft"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir /content/echoheart_demo/output/qwen2-1.5b-merged-ft --output_file /content/echoheart_demo/output/qwen2-1.5b-merged-ft/gguf-model-f16.gguf --out_type f16